# Thema und Hintergrund
Das Hauptziel dieses Projekts ist die Implementierung einer Anwendung, die Github-Tools und Openml-Datensätze ausgeben kann, die den Benutzereingaben entsprechen. Der Benutzer kann eine Beschreibung seiner aktuellen Herausforderung hinzufügen, und das Tool kann entsprechende Informationen zu dieser Herausforderung vorschlagen.

Wir alle wissen, dass das mittlerweile ChatGPT ein starkes Wissen in verschiedenen Bereichen aufweist. Vor allem in Kombination mit der Bing-Suche ist es in der Lage, entsprechende Websites direkt auf Anfrage zu liefern. Aber ChatGPT hat einige Einschränkungen:

Das Wissen von ChatGPT ist auf Daten vor September 2021 trainiert, so dass es keine Möglichkeit gibt, Informationen nach diesem Datum zu liefern.
ChatGPT hat keine Möglichkeit, komplexe logische Beziehungen zu analysieren.
ChatGPT kann keine zitierten Quellen auflisten und seine Verlässlichkeit basiert auf der Verlässlichkeit der Quelleninformationen, die von Natur aus falsch, inkonsistent, falsch oder widersprüchlich sein können, nachdem sie von ChatGPT kombiniert wurden.

Die drei oben genannten Punkte führen dazu, dass es keine Möglichkeit gibt, ChatGPT direkt nach Github Repositories zu fragen.
Deshalb gehen wir wie folgenden Notebook gezeigt vor.

Wir nehemn folgende Aufgabe als Beispiel
“Geht es darum, die Sicherheit von Radwegen zu ermitteln, ist das Forschungspotential riesig. Ziel des SDSC-BW-Projekts war es, dieses zu erkunden. Ob der zahlreichen Problematiken gestaltete sich bereits die Erstellung eines Rahmenwerks schwierig. An erster Stelle standen die vielfältigen, teils verwirrenden Datenquellen – darunter unterschiedliche Websites, die Daten auf ihre eigene Weise abspeichern.”

Das Ergebnis, wenn man die Aufgabe in Bing eingibt:

<img src="images/bing_result.png" alt="feedback from bing search" width="800" height="600">


Das Ergebnis, wenn man die Aufgabe in ChatGPT eingibt:

<div style="display: inline-block;"><img src="images/chatgpt_result.png" alt="feedback from chatgpt chatbot" width="700" height="500"></div>

Wie wir sehen können, gibt gibt ChatGPT keine Webseite zurück, die einen direkten Bezug zu unserem Zweck hat.


Obwohl ChatGPT die Herausforderungen zusammenfassen kann, sind die zur Verfügung gestellten Werkzeuge wenig passen. Darüber hinaus gibt es viele zusätzliche Beschreibungen, die die Antworten kompliziert erscheinen lassen.

# Vorgehen
Um die Probleme von ChatGPT zu lösen, wie z.B. die mangelnde Fähigkeit, komplexe Eingaben zu analysieren, komplexe Antworten, die Unfähigkeit, Echtzeit-Tools bereitzustellen, und mögliche Fehler in den bereitgestellten Links. Unsere Kernideen ist:

1. Die Anforderungen zu zerlegen und jeweils nur einfache Fragen an ChatGPT zu stellen.
2. Die Ausgabe einzuschränken, damit sie kurz und themenbezogen ist.
3. Verwendung der Github-API, um die neuesten Github-Daten zu erhalten, um die Popularität und Effektivität des Tools zu gewährleisten.

Im folgenden zeigen wir im Detail, wie wir vorgehen.

## Packete laden
Zuerst laden wir die benötigten Pakete.

In [ ]:
from data_requests.git_request import search_top_starred_repositories
from data_requests.gpt_request import get_response_from_chatgpt_with_context

import os
import pandas as pd
import requests

"git_request" und "gpt_request" sind benutzerdefinierte Pakete, um über Github eine Liste von Repositories und über Openai eine Liste von Chatgpt-Dienst anzufordern.
"pandas" ist ein Werkzeug zur Datenanalyse. 
"requests" ist ein Werkzeug für den Netzwerkzugriff.

In [ ]:
use_case = input("Bitte gib einen Anwendungsfall ein: ")

# get the theme from the user case
prompt = f"Welches Thema wird in dem folgenden Anwendungsfall untersucht? Bitte antworten Sie nur mit einem Schlüsselwort mit weniger als 20 Buchstaben: {use_case}"
context = []
response, context = get_response_from_chatgpt_with_context(prompt, context)
print(f"\nDas Thema des Anwendungsfalls ist: {response}\n")


Zunächst haben wir ChatGPT das Thema analysieren lassen. Um die Kürze der Antwort zu gewährleisten, haben wir die Ausgabe von ChatGPT durch die folgende Bedingung eingeschränkt.

In [ ]:
# get the challenge from the user case
prompt = f"Welches ist das Hauptproblem, mit dem diese Studie konfrontiert ist? Bitte antworten Sie mit 3 Schlüsselwörtern und ohne Erklärung"
response, context = get_response_from_chatgpt_with_context(prompt, context)
print(f"Die größten Herausforderungen sind: \n{response}\n")

Dann fragen wir ChatGPT nach einer Zusammenfassung der drei Herausforderungen, die sich bei diesem Thema stellen. Da diese Herausforderung auf eine spätere Github-Suche angewandt werden soll, soll ChatGPT, anhand von Schlüsselwörtern und ohne Erklärung antworten.

In [ ]:
# ask for keywords for python tools
prompt = f"Ich möchte nach Python-Tools für das obige Problem nach Schlüsselwörtern suchen, welche Schlüsselwörter sollte ich verwenden, bitte geben Sie mir 3 Vorschläge und trennen Sie sie mit Semikolon, ohne Erklärung"
response, context = get_response_from_chatgpt_with_context(prompt, context)
keywords = response.split(";")

# list the advised git repos
for keyword in keywords:
    prompt = f"Erkläre in weniger als 50 Wörtern warum {keyword} in Kontext von diesem Anwendungsfall gebraucht wird."
    response, context = get_response_from_chatgpt_with_context(prompt, context)
    print(response)

    git_urls, readme_urls = search_top_starred_repositories(keyword+' python')
    if git_urls is not None:
        print("Für diesen Anwendungsfall schlagen wir die folgenden Repositories vor:")
        for git_url, readme_url in zip(git_urls, readme_urls):
            print("Repository URL:", git_url)
            print("README URL:", readme_url)
        print('-'*50)

Auf der Grundlage jedes oben angegebenen Schlüsselworts suchen wir auf dem Github-Server nach passenden Tools und geben die 5 besten Ergebnisse zurück, geordnet nach der Verbreitung dieser Tools. Da die Ergebnisse über den github-Server bereitgestellt werden, ist die Zuverlässigkeit der Ergebnisse gewährleistet. Da wir außerdem nach zusammengefassten Schlüsselwörtern suchen, sind die Ergebnisse eng mit unserem Forschungsthema verbunden.

Die Stichwortsuche basiert auf den folgenden drei Inhalten:
1. Namen und die Beschreibung des Repositorys
2. Quellcode und Dateiinhalt
3. Probleme

Dadurch wird sichergestellt, dass das Repository auch dann durchsucht werden kann, wenn die Schlüsselwörter des Repository-Besitzers falsch gesetzt sind. Da die Schlüsselwörter von chatgpt zusammengefasst werden, ist gewährleistet, dass die erhaltenen Schlüsselwörter keine ungewöhnlichen Wörter enthalten.

# Lokale Anfragen
Natürlich hat das oben beschriebene Verfahren auch Nachteile. Zum Beispiel müssen die Schlüsselwörter im Inhalt des Repositorys erscheinen. Es wir außerdem eine Netzverbindung benötigt.

Das erste Problem kann durch Wordembeddings gelöst werden. Durch das Wordembeddings der Schlüsselwörter und der Readme-Datei des Repositorys können wir das passende Repository finden, indem wir die Ähnlichkeit der beiden Wordembeddings vergleichen. 

Das zweite Problem kann durch den lokalen Einsatz von ChatGPT gelöst werden. Openai's ChatGPT-4 ist noch nicht open source, aber es gibt bereits viele open source Alternativen. Wir können unser Ziel erreichen, indem wir diese Alternativen lokal bereitstellen.

Zu diesem Zweck laden wir zunächst zwei benutzerdefinierte Methoden.

In [ ]:
from data_generation.generate_local_github_database import download_and_save_git_reposiories_according_to_keyword
from data_requests.git_request import search_top_related_local_repositories_with_cs

"download_and_save_git_stared_reposiories" lädt die Repository-Informationen in den lokalen Speicher. "search_top_related_local_repositories" stellt ein lokales ChatGPT bereit und verwendet Wordembeddings, um in der lokalen Datenbank nach passenden Repositories zu suchen.

In [ ]:
# ask for keywords for python tools
prompt = f"Ich möchte nach Python-Tools für das obige Problem nach Schlüsselwörtern suchen, welche Schlüsselwörter sollte ich verwenden, bitte geben Sie mir 3 Vorschläge und trennen Sie sie mit Semikolon, ohne Erklärung"
response, context = get_response_from_chatgpt_with_context(prompt, context)
keywords = response.split(";")

# list the advised git repos
for keyword in keywords:
    prompt = f"Erkläre in weniger als 50 Wörtern warum {keyword} in Kontext von diesem Anwendungsfall gebraucht wird."
    response, context = get_response_from_chatgpt_with_context(prompt, context)
    print(response)

    git_urls, readme_urls = download_and_save_git_reposiories_according_to_keyword(keyword, database_path = './data/repositories.csv')
    if git_urls is not None:
        print("Für diesen Anwendungsfall schlagen wir die folgenden Repositories vor:")
        for git_url, readme_url in zip(git_urls, readme_urls):
            print("Repository URL:", git_url)
            print("README URL:", readme_url)
        print('-'*50)


Wir können sehen, dass es auch lokal gut funktioniert. Da die von uns verwendete Testdatenbank nur zweihundert verschiedene Repositories enthält und eher zentralisiert ist, sehen die Ergebnisse nicht so gut aus wie die Online-Ergebnisse. Dies kann jedoch durch eine Vergrößerung des lokalen Datensatzes behoben werden.